# PyTorch Tutorial 03. Neural Networks

- 基本的に，[このチュートリアル](https://tutorials.pytorch.kr/beginner/blitz/neural_networks_tutorial.html)の内容に基づいている．  
- 徐々にディープラーニングの内容に移っていく


## Define neural network

基礎的なニューラルネットワークを定義する．  
実際学習をさせる前に，以下のような内容を説明するための例題に過ぎない．

- PyTorchではニューラルネットワークをどう定義するか？
- 中ではどういう風に演算がされていくか？
- パディング(padding)がないように見えるが，これはPyTorchの仕様なのか?

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        ############################################################
        ## user-defined layers in the network
        ############################################################
        
        ## define layer (1) convolution (2D)
        ## Conv2d(n_input_channels, n_output_channels, convolution_size)
        self.conv1 = nn.Conv2d(1, 6, 3)  ## (1 channels ->  6 channels) with 3x3 conv
        self.conv2 = nn.Conv2d(6, 16, 3) ## (6 channels -> 16 channels) with 3x3 conv
        
        ## define layer (2) fully connected
        ## Linear(n_input_neurons, n_output_neurons)
        self.fc1 = nn.Linear(16 * 6 * 6, 120) ## (16-channels x 6x6 -> 120)
        self.fc2 = nn.Linear(120, 84)         ## (120 -> 84)
        self.fc3 = nn.Linear(84, 10)          ## ( 84 -> 10)

    def forward(self, x):
        
        ############################################################
        ## user-defined network by connecting layers (NO PADDING!)
        ############################################################
        
        ## block (1) convolution: 32x32x1 -(conv3x3)-> 30x30x6 -(maxpooling)-> 15x15x6
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, (2, 2)) # Max pooling over a (2, 2) window
        
        ## block (2) convolution: 15x15x6 -(conv3x3)-> 13x13x16 -(maxpooling)-> 6x6x16
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2) # size is a square when you only specify a single number
        
        ## top-layer: fully connected network
        x = x.view(-1, self.num_flat_features(x)) ## [N,C,H,W] -> [N, CxHxW]
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
## show model's (trainable) parameters detail
params = list(net.parameters())
print(len(params))

for i in range(len(params)):
    print(params[i].size())

10
torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [3]:
## neural network input & output
input_image = torch.randn(1, 1, 32, 32) ## random image NCHW format
out = net(input_image)
print(out)

tensor([[ 0.0762,  0.0015,  0.0348,  0.1176, -0.0889, -0.1429, -0.0517, -0.1441,
          0.0986,  0.0090]], grad_fn=<AddmmBackward>)


## Loss function

In [4]:
net.zero_grad() ## zero-clear the gradient buffers of all parameters
out.backward(torch.randn(1, 10))

In [5]:
output = net(input_image)
target = torch.randn(10)     # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7931, grad_fn=<MseLossBackward>)


In [6]:
## traceback the autograd functions
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## Backprop

- .backward(): 誤差の逆伝搬(back propagation, backprop)を行う．
- .zero_grad(): 既に計算されていたgradientsをクリアすると思えば良い (?)

In [7]:
net.zero_grad() # zero-clear the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad) # now it will be updated

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0087, -0.0170,  0.0170, -0.0114, -0.0081,  0.0094])


## Update weights

In [10]:
## Stochastic graient in a scratch
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [11]:
import torch.optim as optim

## optimizer for neural network learning
optimizer = optim.SGD(net.parameters(), lr=0.01)

## 
optimizer.zero_grad() # zero the gradient buffers
output = net(input_image)
loss = criterion(output, target)
loss.backward()
optimizer.step()      # Does the update

(end)